In [1]:
# conda install -c conda-forge floweaver

In [2]:
# pip install floweaver

In [3]:
# conda install -c conda-forge ipysankeywidget

In [4]:
# conda install -c conda-forge nodejs

In [5]:
# !jupyter nbextension enable --py --sys-prefix ipysankeywidget

In [6]:
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [7]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix

# Example with code: 
https://towardsdatascience.com/creating-beautiful-sankey-diagrams-with-floweaver-dc1f02fe76bc

In [16]:
import pandas as pd
import numpy as np
from floweaver import *
from  ipysankeywidget import *
from os import *
import shutil
from os.path import isfile, join
import itertools
# from Sankey_code import create_Sankey

#load region and country names 
regs = pd.read_excel('Countries_and_Regions.xlsx', sheet_name = 'regions', index_col=0,engine='openpyxl')['Region']
counts = pd.read_excel('Countries_and_Regions.xlsx', sheet_name = 'names', index_col=0,engine='openpyxl')['Name']

#define path and list all sankey data files
m=getcwd()[0:(len(os.getcwd())-6)]+str('output\\666\\')


####################create itearative sankey images and savethem in a folder
#deleting and creating new folder with sankey images
# if path.exists('Sankey_Images'):
#     shutil.rmtree('Sankey_Images', ignore_errors=False, onerror=None)
# mkdir('Sankey_Images')

year = 2008

Specify the font size:

In [17]:
%%html
<style>
.sankey .node {
    font-size: 28px;
    text-align: center;
}
</style>

In [18]:
counts
#regs

Nr
1                  Austria
2                  Belgium
3           Czech Republic
4                  Germany
5                    Spain
6                  Finland
7                   France
8                   Greece
9                    Italy
10             Netherlands
11                  Poland
12                  Sweden
13                Slovakia
14          United Kingdom
15           United States
16                   Japan
17                   China
18                  Canada
19             South Korea
20                  Brazil
21                   India
22                  Mexico
23                  Russia
24               Australia
25                  Taiwan
26               Indonesia
27    RoW Asia and Pacific
28             RoW America
29              RoW Europe
30                RoW EU27
31              RoW Africa
32         RoW Middle East
Name: Name, dtype: object

# This code creates the Sankeys for the Countries and Regions specified in the counts and regs arrays:

In [29]:
# for region in regs:
region=regs[4]#regions from 1-6
# region = counts[26]#counts from 1-32

#Read PIOT for the specific country
dg = pd.read_excel(str(m)+str(year)+'_Data_for_'+str(region)+'_Sankey.xlsx', sheet_name = 'agg', index_col=0,engine='openpyxl')
dg = dg.round(2)


print(region)



#create empy dataframe, which will be a form of a PIVOT table
d0 = pd.DataFrame()

#run through rows of the Piot
for i in range(0,dg.shape[0]):
    #create empty data frame for temporary saving results
    df = pd.DataFrame()
    
    #save the values of the row
    val_orig = dg.iloc[i,].values
    #save indices where values are not 0 and not NA
    ind = [all(tup) for tup in zip(val_orig!=0, np.isnan(val_orig)==False)]
    
    #save results for value, source & target
    val = val_orig[ind]
    source = [*[dg.index[i]] * len(val)]
    target = dg.columns[ind]

    #save results in the relevant columns
    df['source']=source
    df['target']=target
    df['value']=val

    #append the collected results to the main data frame
    d0=d0.append(df, ignore_index =True)

#add the column 'type' and set all values in it to 'A'    
d0['type']='A'

#change types according to their teal type of flow
d0.at[d0[d0['source'].str.contains("Domestic")].index,'type'] = 'D'
d0.at[d0[d0['source'].str.contains("Foreign")].index,'type'] = 'F'
d0.at[d0[d0['source'].str.contains("Bound")].index,'type'] = 'BI'
d0.at[d0[d0['target'].str.contains("Bound")].index,'type'] = 'BO'
d0.at[d0[d0['source'].str.contains("ROW")].index,'type'] = 'ROW'

#change source names, which means to use the column names, because they do not include the Domestic or Foreign String
for i in dg.columns.values:
    d0.at[d0[d0['source'].str.contains(str(i))].index,'source'] = str(i)

#we change the name of the main data frame to df    
df=d0





#Now, we have to change some names, e.g specify the BO and BI which are now only saved as 'Boundary Outputs' or 'Inputs'
#in the data frame. Further we have to insert also the string '\n' which will be a Zeilenumbruch in the Plot, if the name is
#too long!

#define name-pairs for the boundary outputs (BO), could also be read in from a csv, but for now 
#they can be changed here in the code
BO=[['Mining', 'BO Gangue'],
    ['Reduction','BO Blast furnance\ngas & slag'],
    ['Steelmaking','BO Steelmaking Slag']]

#do the same for the boundary inputs (BI)
BI=[['BI Crude Ore', 'Mining'],
    ['BI Coke/air/flux','Reduction'],
    ['BI Eol-Scrap','Scrap preparation']]

#define also new(=shorter) names pairs Final Outputs strings
FU=[['Construction','Construction'],
    ['Manufacture of machinery and equipment n.e.c.','Machinery and \nequipment n.e.c.'],
   ['Manufacture of motor vehicles, trailers and semi-trailers','Motor vehicles, trailers \nand semi-trailers'],
    ['Manufacturing n.e.c.','Manufacturing n.e.c.'],
   ['Manufacture of other transport equipment','Other transport \nequipment']
    ]

#rename Boundary Output & Inputs names for all relevant rows
for i in range(0,len(BO)):
    df.loc[(df['source']==BO[i][0]) & (df['target']==dg.columns[dg.shape[1]-1]),'target']=BO[i][1]

for i in range(0,len(BI)):
    df.loc[(df['target']==BI[i][1]) & (df['source']==dg.index[dg.shape[0]-1]),'source']=BI[i][0]

















#here we save the detailed final use names
dv_names = pd.read_excel(str(m)+str(year)+'_Data_for_'+str(region)+'_Sankey.xlsx', sheet_name = 'disagg_names', index_col=0,engine='openpyxl',header=None)
dv_names=dv_names.reset_index()

#...and the detailed final use values with their names as a data frames
dv = pd.read_excel(str(m)+str(year)+'_Data_for_'+str(region)+'_Sankey.xlsx', sheet_name = 'disagg', index_col=0,engine='openpyxl')
dv = dv.round(2)
dv=dv.reset_index()
dv.columns=['target','value']
dv['source']=dg.columns[-4]
dv['type']='A'


#...and again change types in the type column
dv.at[dv[dv['target'].str.contains("Domestic")].index,'type'] = 'D'
dv.at[dv[dv['target'].str.contains("Foreign")].index,'type'] = 'F'
dv.at[dv[dv['target'].str.contains("ROW")].index,'type'] = 'ROW'
dv['target']=np.concatenate(np.tile(dv_names[0].values, (3, 1)))
dv=dv[dv['value']!=0]

#here we rename the main data frame for the final use details in db
db=dv

#and we append the detailes final use data frame to the main data frame
df=df.append(db)
#drop Final Use rows because we now have the detailed data appended
df=df[df['target']!='Final Use']

# now we change the detailed final use names to the shorter names we specified above
for i in range(0,len(FU)):
    df.loc[df['target']==FU[i][0],'target']=FU[i][1]

#for indexing we need to reset the index (needs to be sometimes if we append data frames)
df=df.reset_index()
df=df.drop('index',axis=1)#in this process a new column with the old index is created
#but we drop this because it is useless
df=df.round(2)











# create dataframes for the sums we want to display in the final plot
sums=[]
for nam in np.unique(np.append(df['source'].values,df['target'].values)).tolist():
    sums.append(df[df['source']==str(nam)]['value'].sum())

#create he data frame with the aggregates for the flows which we have as sources (almost all)
dx=pd.DataFrame()
dx['flow']=np.unique(np.append(df['source'].values,df['target'].values)).tolist()
dx['value']= sums
dx=dx[dx['value']!=0]
dx=dx.round(2)

#then we do exactly the same for flows we only have as targets, e.g. Final Uses and BO´s
sums1=[]
for nam in np.unique(df['target'].values).tolist():
    sums1.append(df[df['target']==str(nam)]['value'].sum())

dy=pd.DataFrame()
dy['flow']=np.unique(df['target'].values).tolist()
dy['value']= sums1
dy=dy[dy['value']!=0]
dy=dy.round(2)









#define lists of layer dictionaries, layer1='Mining', layer2='Reduction', ...
#this lists are named layer,layer1 and layer2, which is a bit confusing because the first two lists contains the flow-nodes 
#of the plot which are also named layer1,2,3... whereas the list layer2 contains the nodes of the BOs which are named end1,2,3.
#The nodes for the BIs are defined later on.

#this dicts contains next to the name, which is saved in a list, also a title, which is displayed later in the plot. 
#And because we also want the Sum of the Flows displayed, we have this huge complex looking title.
#But this title is basicly the name of the flow and then the sum of the values which are going out of this flow.
#The whole thing looks complicated, becuase the indexing of lists in python is that ugly, especially if we have a 
#further list as index. But we need this strange index lists, because they read out automaticly the correct BOs and BIs,
#and skip the BOs or BIs which are not in the data, to avoid any errors.

#first do all column which are a source in the plot
layers = {'layer'+str(i+1):ProcessGroup(list([dg.columns[i]]), 
            title=dg.columns[i]+'\n'+str(dx[dx['flow']==dg.columns[i]]['value'].values[0])+'\n'
                            ) for i in range(0,len(dg.columns)-2)}

#then we do the same for those which are only a target
layers1 = {'layer'+str(i+len(dg.columns)-1):ProcessGroup(list([[FU[i][1] for i in range(len(FU))][i]]),
            title=[FU[j][1] for j in range(len(FU))][i]+'\n'+str(dy[dy['flow']==[FU[j][1] for j in range(len(FU))][i]]['value'].values[0])+'\n'
            ) for i in range(0,len(np.unique(db['target'].values).tolist()))}


#and finally the BO nodes
BO_ind=[[v[1] for v in BO].index([ e for e in [v[1] for v in BO] if e in dy['flow'].values ][z]) for z in range(len([ e for e in [v[1] for v in BO] if e in dy['flow'].values ]))]
layers2 = {'end'+str(i+1):ProcessGroup(list([[v[1] for v in BO][i]]), title=[v[1] for v in BO][i]+'\n'+str(dy[dy['flow']==[v[1] for v in BO][i]]['value'].values[0])+'\n')
          for i in BO_ind}


#get columns/flows which have flows to the scrap node:
scrap_flows = df[(df['target'].str.contains('Scrap')) & (df['type']!='BI')]['source'].values
#get the name of the scrap layer, becuase we need that often
scrap_layer = list(layers.keys())[np.where(dg.columns.str.contains('Scrap'))[0][0]]
#get layer-endings(=the number at the end of the string'layer1') & full layer names which have flows to scrap node
layer_endings=[list(layers.keys())[np.where(dg.columns==scrap_flows[i])[0][0]][-1] for i in range(len(scrap_flows))]
layer_fullnames=[list(layers.keys())[np.where(dg.columns==scrap_flows[i])[0][0]] for i in range(len(scrap_flows))]


##############Ordering: here we specify the order of the nodes, starting with some fixed positions
#first col: 1st start layer
#beginning with the BI curde Ore node
a=[['start1'],[],[]]

#second col: followed by Mining and the other start layers
b=[['layer1'],['start2'],['start3']] 

#third col:first end layer the reduction layer and the scrap layer
c=[['end1'],['layer2',scrap_layer],[]] 

#define empty ordering list which we fill with the predefined columns
ordering=list()
ordering=ordering+[a]+[b]+[c]

#layers inbetween:
#make empty list for waypoints:
waypoints=list({'w1': Waypoint(direction='L', title='')}.items())+list({'ws5': Waypoint(direction='L', title='')}.items())+list({'ws6': Waypoint(direction='L', title='')}.items())


#for i in all other layer endings except the scrap layer ending
j=1
for i in [x for x in [v[-1] for v in list(layers.keys())[2:]] if (x!=scrap_layer[-1])]:#&(x!=Final_use_layer[-1])]: 
    o = [[],['layer'+str(i)],[]]#if no further if conditions are fulfilled then this is the next ordering column
    if j<len(BO):#if we have not all end(=BOs) nodes placed,...
        j=j+1
        o = [['end'+str(j)],['layer'+str(i)],[]]#...then we have to place them next to the current layer we are looking at
        
        #but if we are looking at the first layer after the scrap layer, we also place a waypoint w1 to have a better plot:
        if i==[x for x in [v[-1] for v in list(layers.keys())[2:]] if (x!=scrap_layer[-1])][0]:
            o = [['end'+str(j)],['layer'+str(i)],['w1']]

        #and if we have a layer in the current ordering column, where the layer in the next ordering column has a flow
        #back to the scrap layer, we have to set here another waypoint for a better plot:
        if int(i)+1 in [int(numeric_string)+1 for numeric_string in layer_endings]:  
            o = [['end'+str(j)],['layer'+str(i)],['w'+str(int(i)+1)]]
        #we also save this waypoint in the waypoint list
        waypoints=waypoints+list({'w'+str(int(i)+1): Waypoint(direction='L', title='')}.items())
    
    #...and if we have already all ends played, we only have to playe the waypoints if the layer in the next ordering
    #column has a flow back to the scrap layer
    else:
        if int(i)+1 in [int(numeric_string)+1 for numeric_string in layer_endings]:  
            o = [[],['layer'+str(i)],['w'+str(int(i)+1)]]
        #we also save this waypoint in the waypoint list
        waypoints=waypoints+list({'w'+str(int(i)+1): Waypoint(direction='L', title='')}.items())
    #then we append the ordering column
    ordering=ordering+[o]

#last col: the 5 Final Uses detailed are specified
d=[[],['layer'+str(i+len(dg.columns)-1) for i in range(0,len(np.unique(db['target'].values).tolist()))],[]]

#and then add the last column
ordering=ordering+[d]
print(ordering)












#####################################################   NODES: 
#now we define the BI nodes, start 1,2,3
nodes = dict(list({'start'+str(i): ProcessGroup(list([[v[0] for v in BI][i-1]]), title=[v[0] for v in BI][i-1]) for i in [1,2,3]}.items())+
     list({'end'+str(i+1): ProcessGroup(list([[v[1] for v in BO][i]]), title=[v[1] for v in BO][i]) for i in BO_ind}.items()))

#now we save which of the 3 BOs is missing in the data and skip this node
missing_list = list(set(['end1','end2', 'end3'])-set(list(layers2.keys())))
missing_ends = []
if missing_list!=[]:
    missing_ends = list({'end'+i[3]:ProcessGroup(list('Missing')) for i in list(set(['end1','end2', 'end3'])-set(list(layers2.keys())))}.items())

#here we add the nodes in the node list together
nodes = dict(list(nodes.items()) + list(layers.items()) + list(layers1.items()) + list(layers2.items()) + waypoints + missing_ends)

#####################################################   BUNDLES:
#the bundles define which cnnections between the nodes should be displayed, and it is forbidden to include duplicates.
#Also one can specify if waypoints are included in the flow and the direction of the flow.

#define a empty list and get the boolean list of indexes in which ordering column we have an waypoint
waypointsindex = list([])
for y in [[x for x in a][2] for a in ordering]:
    if len(y)>0:
        if 'w' in y[0]:
            waypointsindex=waypointsindex+[True]
        else: 
            waypointsindex=waypointsindex+[False]
    else:
        waypointsindex=waypointsindex+[False]

#save the layers which have a waypoint(except 'w1') in their column ...
wp_layers = [[[x for x in a][1] for a in ordering][y] for y in (np.asarray(([x for x in [np.where(waypointsindex)]][0]))[0]).tolist()][1:]
#...and the names of this waypoints
wp_waypoints = [[[x for x in a][2] for a in ordering][y] for y in (np.asarray(([x for x in [np.where(waypointsindex)]][0]))[0]).tolist()][1:]

wp_strings = list(['w1'])
waypointstrings= list([])

#if we have several waypoints the the flow should run through them in reversed order, from right to left: [w7,w6,w5,w1]
for i in range(len(wp_layers)):
    wp_strings=[wp_waypoints[i][0]]+wp_strings
    waypointstrings=waypointstrings+[wp_strings]

#now we add all teh relevant bundles to gether in a huge list:
bundles = [Bundle('layer'+str(i), 'layer'+str(i+1)) for i in [v+1 for v in range(len(dg.columns)-4)]]+\
[Bundle(scrap_layer, 'layer'+str(i)) for i in [v+1 for v in range(len(dg.columns)-3)]]+\
list(itertools.chain.from_iterable([[Bundle('start'+str(j+1),'layer'+str(i)) for i in [v+1 for v in range(len(dg.columns)-1)]] for j in range(len(BI))]))+\
list(itertools.chain.from_iterable([[Bundle('layer'+str(i),'end'+str(j+1)) for i in [v+1 for v in range(len(dg.columns)-1)]] for j in BO_ind]))+\
[Bundle('layer'+str(len(dg.columns)-3),'layer'+str(len(dg.columns)-2+i)) for i in range(1,len(np.unique(db['target'].values).tolist())+1)]+\
[Bundle(wp_layers[i][0], scrap_layer, waypointstrings[i]) for i in range(len(wp_layers))]+\
[Bundle(source='layer4', target='layer3', waypoints=(), flow_selection=None, flow_partition=None, default_partition=None),
 Bundle(source='layer5', target='layer3', waypoints=(), flow_selection=None, flow_partition=None, default_partition=None)]


#####################################################    PLOT:
# add the partitions which determine the column partition
partition_type = Dataset(df).partition('type')

# add the color palette
palette = {'D': '#481567',#'#FF00FF',#'#440154FF', #'#541352',
           'F': '#FDE725', #'#FFD700',
          'BI': '#F88379',#'#10a53d',
          'BO': '#F5F5F5',
          'ROW': '#00FF00'}


# create the sankey diagram
sdd = SankeyDefinition(nodes, bundles, ordering, 
                       flow_partition=partition_type)

# display & save the sankey diagram
weave(sdd, df, palette=palette
     ).to_widget(width=2680, height=720, margins=dict(left=220, right=320),align_link_types=False).auto_save_png(str(m)+'Sankey_Images\\'+str(year)+'_'+str(region)+'.png')

China
[[['start1'], [], []], [['layer1'], ['start2'], ['start3']], [['end1'], ['layer2', 'layer7'], []], [['end2'], ['layer3'], ['w1']], [['end3'], ['layer4'], []], [[], ['layer5'], []], [[], ['layer6'], ['w7']], [[], ['layer8', 'layer9', 'layer10', 'layer11', 'layer12'], []]]


SankeyWidget(groups=[{'id': '__w1_layer7_2', 'type': 'group', 'title': '', 'nodes': ['__w1_layer7_2^*']}, {'id…

# Here is almost the same Code which creates the sankey for the World and World-without-China Sankeys:

In [107]:
# for region in regs:

region='World'
region='World-without-China'
dg = pd.read_excel('./Sankey_Data/'+str(year)+'_Data_for_'+str(region)+'_Sankey.xlsx', sheet_name = 'agg', index_col=0,engine='openpyxl')
dg = dg.round(2)


print(region)




d0 = pd.DataFrame()

for i in range(0,dg.shape[0]):
    df = pd.DataFrame()

    val_orig = dg.iloc[i,].values
    ind = [all(tup) for tup in zip(val_orig!=0, np.isnan(val_orig)==False)]

    val = val_orig[ind]
    source = [*[dg.index[i]] * len(val)]
    target = dg.columns[ind]

    df['source']=source
    df['target']=target
    df['value']=val

    d0=d0.append(df, ignore_index =True)

d0['type']='A'

#change types
d0.at[d0[d0['source'].str.contains("Domestic")].index,'type'] = 'D'
d0.at[d0[d0['source'].str.contains("Foreign")].index,'type'] = 'F'
d0.at[d0[d0['source'].str.contains("Bound")].index,'type'] = 'BI'
d0.at[d0[d0['target'].str.contains("Bound")].index,'type'] = 'BO'
d0.at[d0[d0['source'].str.contains("ROW")].index,'type'] = 'ROW'

#change source names
for i in dg.columns.values:
    d0.at[d0[d0['source'].str.contains(str(i))].index,'source'] = str(i)

df=d0

#define name-pairs for the boundary outputs (BO), could also be read in from a csv
BO=[['Mining', 'BO Gangue'],
    ['Reduction','BO Blast furnance\ngas & slag'],
    ['Steelmaking','BO Steelmaking Slag']]

#do the same for the boundary inputs (BI)
BI=[['BI Crude Ore', 'Mining'],
    ['BI Coke/air/flux','Reduction'],
    ['BI Eol-Scrap','Scrap preparation']]

#define Final Outputs strings
FU=[['Construction','Construction'],
    ['Manufacture of machinery and equipment n.e.c.','Machinery and \nequipment n.e.c.'],
   ['Manufacture of motor vehicles, trailers and semi-trailers','Motor vehicles, trailers \nand semi-trailers'],
    ['Manufacturing n.e.c.','Manufacturing n.e.c.'],
   ['Manufacture of other transport equipment','Other transport \nequipment']
    ]
#define further nodes which all have no next node
ends=['Final Use']

#rename Boundary Output & Inputs names for all relevant columns
for i in range(0,len(BO)):
    df.loc[(df['source']==BO[i][0]) & (df['target']==dg.columns[dg.shape[1]-1]),'target']=BO[i][1]

for i in range(0,len(BI)):
    df.loc[(df['target']==BI[i][1]) & (df['source']==dg.index[dg.shape[0]-1]),'source']=BI[i][0]


















dv_names = pd.read_excel('./Sankey_Data/'+str(year)+'_Data_for_'+str(region)+'_Sankey.xlsx', sheet_name = 'disagg_names', index_col=0,engine='openpyxl',header=None)
dv_names=dv_names.reset_index()

dv = pd.read_excel('./Sankey_Data/'+str(year)+'_Data_for_'+str(region)+'_Sankey.xlsx', sheet_name = 'disagg', index_col=0,engine='openpyxl')
dv = dv.round(2)
dv=dv.reset_index()
dv.columns=['target','value']

dv['source']=dg.columns[-4]
dv['type']='A'


#change types
dv.at[dv[dv['target'].str.contains("Domestic")].index,'type'] = 'D'
dv.at[dv[dv['target'].str.contains("Foreign")].index,'type'] = 'F'
dv['target']=np.concatenate(np.tile(dv_names[0].values, (2, 1)))
dv=dv[dv['value']!=0]


db=dv


df=df.append(db)
# #drop Final Use rows
df=df[df['target']!='Final Use']

# df[[all(tup) for tup in zip(df['source']==dg.columns[-4], df['target'].str.contains('Scrap')==False)]]
for i in range(0,len(FU)):
    df.loc[df['target']==FU[i][0],'target']=FU[i][1]

df=df.reset_index()
df=df.drop('index',axis=1)
df=df.round(2)











# create dataframes for the sums
sums=[]
for nam in np.unique(np.append(df['source'].values,df['target'].values)).tolist():
    sums.append(df[df['source']==str(nam)]['value'].sum())

sums
dx=pd.DataFrame()
dx['flow']=np.unique(np.append(df['source'].values,df['target'].values)).tolist()
dx['value']= sums
dx=dx[dx['value']!=0]
dx=dx.round(2)
sums1=[]
for nam in np.unique(df['target'].values).tolist():
    sums1.append(df[df['target']==str(nam)]['value'].sum())

sums1
dy=pd.DataFrame()
dy['flow']=np.unique(df['target'].values).tolist()
dy['value']= sums1
dy=dy[dy['value']!=0]
dy=dy.round(2)









#define layers
#first do all column which are no final node
layers = {'layer'+str(i+1):ProcessGroup(list([dg.columns[i]]), 
            title=dg.columns[i]+'\n'+str(dx[dx['flow']==dg.columns[i]]['value'].values[0])+'\n'
                            ) for i in range(0,len(dg.columns)-2)}

layers1 = {'layer'+str(i+len(dg.columns)-1):ProcessGroup(list([[FU[i][1] for i in range(len(FU))][i]]),
            title=[FU[j][1] for j in range(len(FU))][i]+'\n'+str(dy[dy['flow']==[FU[j][1] for j in range(len(FU))][i]]['value'].values[0])+'\n'
            ) for i in range(0,len(np.unique(db['target'].values).tolist()))}


#and finally the BO nodes
BO_ind=[[v[1] for v in BO].index([ e for e in [v[1] for v in BO] if e in dy['flow'].values ][z]) for z in range(len([ e for e in [v[1] for v in BO] if e in dy['flow'].values ]))]
layers2 = {'end'+str(i+1):ProcessGroup(list([[v[1] for v in BO][i]]), title=[v[1] for v in BO][i]+'\n'+str(dy[dy['flow']==[v[1] for v in BO][i]]['value'].values[0])+'\n')
          for i in BO_ind}


#get columns which have flows to scrap:
scrap_flows = df[(df['target'].str.contains('Scrap')) & (df['type']!='BI')]['source'].values
#get layer-endings & full names which have flows to scrap
scrap_layer = list(layers.keys())[np.where(dg.columns.str.contains('Scrap'))[0][0]]
layer_endings=[list(layers.keys())[np.where(dg.columns==scrap_flows[i])[0][0]][-1] for i in range(len(scrap_flows))]
layer_fullnames=[list(layers.keys())[np.where(dg.columns==scrap_flows[i])[0][0]] for i in range(len(scrap_flows))]


##############Ordering code with waypoints!!!!!:
#first col: 1st start layer
a=[['start1'],[],[]]

#second col: first layer and other start layers
# if scrap layer exists:...
b=[['layer1'],['start2'],['start3']] 

#third col:first end layer
c=[['end1'],['layer2',scrap_layer],[]] 

#define ordering list
ordering=list()
ordering=ordering+[a]+[b]+[c]
#layers inbetween:
#make empty list for waypoints:
waypoints=list({'w1': Waypoint(direction='L', title='')}.items())+list({'ws5': Waypoint(direction='L', title='')}.items())+list({'ws6': Waypoint(direction='L', title='')}.items())


#for i in all other layer endings except the scrap layer ending
j=1
for i in [x for x in [v[-1] for v in list(layers.keys())[2:]] if (x!=scrap_layer[-1])]:#&(x!=Final_use_layer[-1])]: 
    o = [[],['layer'+str(i)],[]]
    if j<len(BO):
        j=j+1
        o = [['end'+str(j)],['layer'+str(i)],[]]
        if i==[x for x in [v[-1] for v in list(layers.keys())[2:]] if (x!=scrap_layer[-1])][0]:
            o = [['end'+str(j)],['layer'+str(i)],['w1']]

        if int(i)+1 in [int(numeric_string)+1 for numeric_string in layer_endings]:  
            o = [['end'+str(j)],['layer'+str(i)],['w'+str(int(i)+1)]]
        waypoints=waypoints+list({'w'+str(int(i)+1): Waypoint(direction='L', title='')}.items())
    else:
        if int(i)+1 in [int(numeric_string)+1 for numeric_string in layer_endings]:  
            o = [[],['layer'+str(i)],['w'+str(int(i)+1)]]
        waypoints=waypoints+list({'w'+str(int(i)+1): Waypoint(direction='L', title='')}.items())
    ordering=ordering+[o]
#     print(o)

#last col: Final Use
d=[[],['layer'+str(i+len(dg.columns)-1) for i in range(0,len(np.unique(db['target'].values).tolist()))],[]]

ordering=ordering+[d]













#####################################################   NODES:

nodes = dict(list({'start'+str(i): ProcessGroup(list([[v[0] for v in BI][i-1]]), title=[v[0] for v in BI][i-1]) for i in [1,2,3]}.items())+
     list({'end'+str(i+1): ProcessGroup(list([[v[1] for v in BO][i]]), title=[v[1] for v in BO][i]) for i in BO_ind}.items()))

missing_list = list(set(['end1','end2', 'end3'])-set(list(layers2.keys())))
missing_ends = []
if missing_list!=[]:
    missing_ends = list({'end'+i[3]:ProcessGroup(list('Missing')) for i in list(set(['end1','end2', 'end3'])-set(list(layers2.keys())))}.items())

nodes = dict(list(nodes.items()) + list(layers.items()) + list(layers1.items()) + list(layers2.items()) + waypoints + missing_ends)

#####################################################   BUNDLES:

# set the "bundle" of connections you want to show
import itertools
waypointsindex = list([])
for y in [[x for x in a][2] for a in ordering]:
    if len(y)>0:
        if 'w' in y[0]:
            waypointsindex=waypointsindex+[True]
        else: 
            waypointsindex=waypointsindex+[False]
    else:
        waypointsindex=waypointsindex+[False]

wp_layers = [[[x for x in a][1] for a in ordering][y] for y in (np.asarray(([x for x in [np.where(waypointsindex)]][0]))[0]).tolist()][1:]
wp_waypoints = [[[x for x in a][2] for a in ordering][y] for y in (np.asarray(([x for x in [np.where(waypointsindex)]][0]))[0]).tolist()][1:]

wp_strings = list(['w1'])
waypointstrings= list([])
for i in range(len(wp_layers)):
    wp_strings=[wp_waypoints[i][0]]+wp_strings
    waypointstrings=waypointstrings+[wp_strings]

bundles = [Bundle('layer'+str(i), 'layer'+str(i+1)) for i in [v+1 for v in range(len(dg.columns)-4)]]+\
[Bundle(scrap_layer, 'layer'+str(i)) for i in [v+1 for v in range(len(dg.columns)-3)]]+\
list(itertools.chain.from_iterable([[Bundle('start'+str(j+1),'layer'+str(i)) for i in [v+1 for v in range(len(dg.columns)-1)]] for j in range(len(BI))]))+\
list(itertools.chain.from_iterable([[Bundle('layer'+str(i),'end'+str(j+1)) for i in [v+1 for v in range(len(dg.columns)-1)]] for j in BO_ind]))+\
[Bundle('layer'+str(len(dg.columns)-3),'layer'+str(len(dg.columns)-2+i)) for i in range(1,len(np.unique(db['target'].values).tolist())+1)]+\
[Bundle(wp_layers[i][0], scrap_layer, waypointstrings[i]) for i in range(len(wp_layers))]+\
[Bundle(source='layer4', target='layer3', waypoints=(), flow_selection=None, flow_partition=None, default_partition=None),
 Bundle(source='layer5', target='layer3', waypoints=(), flow_selection=None, flow_partition=None, default_partition=None)]

#####################################################    PLOT:
# add the partitions
begins = Partition.Simple('source', df['source'].unique())
partition_type = Dataset(df).partition('type')

# nodes['start'].partition = begins
# nodes['end'].partition = Partition.Simple('target', 
#                          df['target'].unique())
# add the color palette
palette = {'D': '#481567',#'#FF00FF',#'#440154FF', #'#541352',
           'F': '#FDE725', #'#FFD700',
          'BI': '#F88379',#'#10a53d',
          'BO': '#F5F5F5',
          'ROW': '#00FF00'}


# create the sankey diagram
sdd = SankeyDefinition(nodes, bundles, ordering, 
                       flow_partition=partition_type)

# display & save the sankey diagram
weave(sdd, df, palette=palette
     ).to_widget(width=2680, height=720, margins=dict(left=220, right=320),align_link_types=False).auto_save_png('./Sankey_Images/'+str(year)+'_'+str(region)+'.png')

World-without-China


SankeyWidget(groups=[{'id': '__w1_layer7_2', 'type': 'group', 'title': '', 'nodes': ['__w1_layer7_2^*']}, {'id…